In [1]:
from PIL import Image
from keras.models import Model
from keras.layers import Dense
from keras.layers import Flatten
from keras.models import Sequential, load_model
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.resnet50 import preprocess_input
from keras.applications.resnet50 import decode_predictions
import os
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
import joblib
import cv2
from keras.layers import Dense, Embedding, BatchNormalization
from keras.utils import to_categorical
from keras.layers import Input, Flatten, Dropout, Activation
from keras.layers import Conv1D, MaxPooling1D
from keras.models import Model

In [2]:
path = []
label = []
for i in os.listdir('mfcc_audio/'):
    path.append('mfcc_audio/'+i)
    label.append(i[7:8])
df = pd.DataFrame({'path' : path, 'label': label})

In [3]:
len(df)

13718

In [4]:
df.head()

,path,label
0,mfcc_audio/03-01-01-01-01-01-01.wav.wav,1
1,mfcc_audio/03-01-01-01-01-01-02.wav.wav,1
2,mfcc_audio/03-01-01-01-01-01-03.wav.wav,1
3,mfcc_audio/03-01-01-01-01-01-04.wav.wav,1
4,mfcc_audio/03-01-01-01-01-01-05.wav.wav,1


In [5]:
df

,path,label
0,mfcc_audio/03-01-01-01-01-01-01.wav.wav,1
1,mfcc_audio/03-01-01-01-01-01-02.wav.wav,1
2,mfcc_audio/03-01-01-01-01-01-03.wav.wav,1
3,mfcc_audio/03-01-01-01-01-01-04.wav.wav,1
4,mfcc_audio/03-01-01-01-01-01-05.wav.wav,1
...,...,...
13713,mfcc_audio/98-94-06-17-62-41-96.wav.wav,6
13714,mfcc_audio/98-95-01-23-22-24-20.wav.wav,1
13715,mfcc_audio/98-95-04-69-80-64-68.wav,4
13716,mfcc_audio/98-95-05-11-45-65-78.wav.wav,5


In [6]:
train, test = train_test_split(df, test_size=0.2)

In [7]:
train

,path,label
12447,mfcc_audio/89-85-05-49-65-24-92.wav,5
615,mfcc_audio/03-01-04-02-01-02-16.wav.wav,4
11402,mfcc_audio/82-75-05-47-94-55-26.wav,5
1447,mfcc_audio/12-96-05-14-41-31-19.wav.wav,5
5499,mfcc_audio/41-27-04-53-12-39-29.wav,4
...,...,...
2793,mfcc_audio/22-47-01-17-54-86-97.wav,1
6651,mfcc_audio/49-35-01-52-48-38-39.wav,1
1418,mfcc_audio/12-71-05-39-73-74-32.wav,5
5064,mfcc_audio/38-23-05-33-79-81-92.wav.wav,5


In [8]:
test

,path,label
5555,mfcc_audio/41-63-03-54-56-92-81.wav,3
2287,mfcc_audio/18-87-05-12-50-20-75.wav,5
11546,mfcc_audio/83-88-02-87-35-27-62.wav,2
7717,mfcc_audio/56-93-06-84-72-91-69.wav,6
3238,mfcc_audio/25-64-03-28-16-59-11.wav,3
...,...,...
10255,mfcc_audio/74-76-04-71-24-15-48.wav.wav,4
4513,mfcc_audio/34-37-02-35-15-10-63.wav.wav,2
10302,mfcc_audio/75-20-04-65-12-28-39.wav,4
12381,mfcc_audio/89-51-04-54-29-96-64.wav.wav,4


In [9]:
mel_train = train.copy()

In [10]:
mel_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10974 entries, 12447 to 12514
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   path    10974 non-null  object
 1   label   10974 non-null  object
dtypes: object(2)
memory usage: 257.2+ KB


In [11]:
mel_train.to_csv('mel_train_mfcc.csv', index = False)

In [18]:
mel_train_df = pd.read_csv('mel_train_mfcc.csv')

In [19]:
mel_train_df.head()

,path,label
0,mfcc_audio/89-85-05-49-65-24-92.wav,5
1,mfcc_audio/03-01-04-02-01-02-16.wav.wav,4
2,mfcc_audio/82-75-05-47-94-55-26.wav,5
3,mfcc_audio/12-96-05-14-41-31-19.wav.wav,5
4,mfcc_audio/41-27-04-53-12-39-29.wav,4


In [20]:
for i in range(len(mel_train_df)):
    mel_train_df.path[i] = 'data_cropped/'+mel_train_df.path[i].split('/')[1].split('.')[0]+'.png'

<ipython-input-20-3d29efccdbee>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mel_train_df.path[i] = 'data_cropped/'+mel_train_df.path[i].split('/')[1].split('.')[0]+'.png'


In [21]:
mel_train_df

,path,label
0,data_cropped/89-85-05-49-65-24-92.png,5
1,data_cropped/03-01-04-02-01-02-16.png,4
2,data_cropped/82-75-05-47-94-55-26.png,5
3,data_cropped/12-96-05-14-41-31-19.png,5
4,data_cropped/41-27-04-53-12-39-29.png,4
...,...,...
10969,data_cropped/22-47-01-17-54-86-97.png,1
10970,data_cropped/49-35-01-52-48-38-39.png,1
10971,data_cropped/12-71-05-39-73-74-32.png,5
10972,data_cropped/38-23-05-33-79-81-92.png,5


In [22]:
train.to_csv('mfcc_audio_train.csv', index = False)

In [23]:
mfcc_df_train = pd.read_csv('mfcc_audio_train.csv')
mfcc_df_train

,path,label
0,mfcc_audio/89-85-05-49-65-24-92.wav,5
1,mfcc_audio/03-01-04-02-01-02-16.wav.wav,4
2,mfcc_audio/82-75-05-47-94-55-26.wav,5
3,mfcc_audio/12-96-05-14-41-31-19.wav.wav,5
4,mfcc_audio/41-27-04-53-12-39-29.wav,4
...,...,...
10969,mfcc_audio/22-47-01-17-54-86-97.wav,1
10970,mfcc_audio/49-35-01-52-48-38-39.wav,1
10971,mfcc_audio/12-71-05-39-73-74-32.wav,5
10972,mfcc_audio/38-23-05-33-79-81-92.wav.wav,5


In [24]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2744 entries, 5555 to 499
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   path    2744 non-null   object
 1   label   2744 non-null   object
dtypes: object(2)
memory usage: 64.3+ KB


In [25]:
mel_test = test.copy()

In [26]:
mel_test

,path,label
5555,mfcc_audio/41-63-03-54-56-92-81.wav,3
2287,mfcc_audio/18-87-05-12-50-20-75.wav,5
11546,mfcc_audio/83-88-02-87-35-27-62.wav,2
7717,mfcc_audio/56-93-06-84-72-91-69.wav,6
3238,mfcc_audio/25-64-03-28-16-59-11.wav,3
...,...,...
10255,mfcc_audio/74-76-04-71-24-15-48.wav.wav,4
4513,mfcc_audio/34-37-02-35-15-10-63.wav.wav,2
10302,mfcc_audio/75-20-04-65-12-28-39.wav,4
12381,mfcc_audio/89-51-04-54-29-96-64.wav.wav,4


In [28]:
mel_test.to_csv('mel_test_mfcc.csv', index =False)

In [29]:
mel_test_df =  pd.read_csv('mel_test_mfcc.csv')
mel_test_df

,path,label
0,mfcc_audio/41-63-03-54-56-92-81.wav,3
1,mfcc_audio/18-87-05-12-50-20-75.wav,5
2,mfcc_audio/83-88-02-87-35-27-62.wav,2
3,mfcc_audio/56-93-06-84-72-91-69.wav,6
4,mfcc_audio/25-64-03-28-16-59-11.wav,3
...,...,...
2739,mfcc_audio/74-76-04-71-24-15-48.wav.wav,4
2740,mfcc_audio/34-37-02-35-15-10-63.wav.wav,2
2741,mfcc_audio/75-20-04-65-12-28-39.wav,4
2742,mfcc_audio/89-51-04-54-29-96-64.wav.wav,4


In [30]:
for i in range(len(mel_test_df)):
    mel_test_df.path[i] = 'data_cropped/'+mel_test_df.path[i].split('/')[1].split('.')[0]+'.png'

<ipython-input-30-21c289361d0a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mel_test_df.path[i] = 'data_cropped/'+mel_test_df.path[i].split('/')[1].split('.')[0]+'.png'


In [31]:
mel_test_df

,path,label
0,data_cropped/41-63-03-54-56-92-81.png,3
1,data_cropped/18-87-05-12-50-20-75.png,5
2,data_cropped/83-88-02-87-35-27-62.png,2
3,data_cropped/56-93-06-84-72-91-69.png,6
4,data_cropped/25-64-03-28-16-59-11.png,3
...,...,...
2739,data_cropped/74-76-04-71-24-15-48.png,4
2740,data_cropped/34-37-02-35-15-10-63.png,2
2741,data_cropped/75-20-04-65-12-28-39.png,4
2742,data_cropped/89-51-04-54-29-96-64.png,4


In [32]:
test.to_csv('mfcc_audio_test.csv', index = False)

In [33]:
mfcc_df_test = pd.read_csv('mfcc_audio_test.csv')

In [34]:
mfcc_df_test

,path,label
0,mfcc_audio/41-63-03-54-56-92-81.wav,3
1,mfcc_audio/18-87-05-12-50-20-75.wav,5
2,mfcc_audio/83-88-02-87-35-27-62.wav,2
3,mfcc_audio/56-93-06-84-72-91-69.wav,6
4,mfcc_audio/25-64-03-28-16-59-11.wav,3
...,...,...
2739,mfcc_audio/74-76-04-71-24-15-48.wav.wav,4
2740,mfcc_audio/34-37-02-35-15-10-63.wav.wav,2
2741,mfcc_audio/75-20-04-65-12-28-39.wav,4
2742,mfcc_audio/89-51-04-54-29-96-64.wav.wav,4


In [35]:
import librosa

In [37]:
import time
import os
lst = []

start_time = time.time()

for lab, audio in enumerate(mfcc_df_train.path):
    try:
        #Load librosa array, obtain mfcss, store the file and the mcss information in a new array
        X, sample_rate = librosa.load(audio, res_type='kaiser_fast')
        mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T,axis=0) 
        # The instruction below converts the labels (from 1 to 8) to a series from 0 to 7
        # This is because our predictor needs to start from 0 otherwise it will try to predict also 0.
        file = int(mfcc_df_train.label[lab]) - 1 
        arr = mfccs, file
        lst.append(arr)
        print(audio, lab, mfcc_df_train.label[lab])
        # If the file is not valid, skip it
    except ValueError:
        continue

print("--- Data loaded. Loading time: %s seconds ---" % (time.time() - start_time))

mfcc_audio/89-85-05-49-65-24-92.wav 0 5
mfcc_audio/03-01-04-02-01-02-16.wav.wav 1 4
mfcc_audio/82-75-05-47-94-55-26.wav 2 5
mfcc_audio/12-96-05-14-41-31-19.wav.wav 3 5
mfcc_audio/41-27-04-53-12-39-29.wav 4 4
mfcc_audio/29-38-04-70-77-19-39.wav.wav 5 4
mfcc_audio/84-33-01-75-86-23-70.wav 6 1
mfcc_audio/49-66-03-68-98-57-52.wav 7 3
mfcc_audio/87-56-05-89-10-59-16.wav 8 5
mfcc_audio/69-14-03-37-18-95-79.wav.wav 9 3
mfcc_audio/57-24-05-69-29-90-66.wav 10 5
mfcc_audio/95-39-05-17-93-69-68.wav 11 5
mfcc_audio/65-78-06-54-20-81-91.wav 12 6
mfcc_audio/35-10-04-36-62-58-91.wav.wav 13 4
mfcc_audio/59-28-02-97-26-93-35.wav 14 2
mfcc_audio/50-57-04-84-59-82-81.wav 15 4
mfcc_audio/85-41-05-11-17-24-22.wav 16 5
mfcc_audio/40-67-06-64-82-69-44.wav 17 6
mfcc_audio/71-43-06-79-36-45-44.wav 18 6
mfcc_audio/17-90-01-79-81-88-20.wav 19 1
mfcc_audio/25-17-06-73-39-19-18.wav 20 6
mfcc_audio/67-58-06-17-64-10-76.wav.wav 21 6
mfcc_audio/88-64-02-48-43-90-16.wav 22 2
mfcc_audio/12-22-03-82-26-83-24.wav 23 3
mf

mfcc_audio/87-50-06-79-14-70-76.wav 193 6
mfcc_audio/24-28-06-92-50-81-11.wav 194 6
mfcc_audio/14-28-05-42-60-63-81.wav 195 5
mfcc_audio/34-80-05-63-83-97-79.wav 196 5
mfcc_audio/42-61-06-44-54-16-43.wav 197 6
mfcc_audio/58-67-05-43-25-55-45.wav 198 5
mfcc_audio/10-57-04-24-61-45-74.wav.wav 199 4
mfcc_audio/67-93-06-44-80-58-89.wav 200 6
mfcc_audio/60-74-02-17-19-62-54.wav 201 2
mfcc_audio/84-80-03-15-19-52-79.wav 202 3
mfcc_audio/39-10-03-47-44-20-25.wav 203 3
mfcc_audio/34-79-06-50-91-11-36.wav 204 6
mfcc_audio/51-94-03-71-45-55-44.wav 205 3
mfcc_audio/80-84-05-59-18-57-22.wav 206 5
mfcc_audio/53-45-04-90-89-66-96.wav 207 4
mfcc_audio/75-51-05-24-60-40-58.wav 208 5
mfcc_audio/03-01-02-01-02-01-17.wav.wav 209 2
mfcc_audio/37-54-02-36-27-82-94.wav 210 2
mfcc_audio/24-39-05-11-90-67-98.wav 211 5
mfcc_audio/50-62-02-66-63-70-24.wav 212 2
mfcc_audio/28-18-05-13-89-62-73.wav 213 5
mfcc_audio/39-83-04-40-69-37-47.wav.wav 214 4
mfcc_audio/34-58-04-93-63-11-52.wav 215 4
mfcc_audio/54-50-04-44

mfcc_audio/81-68-06-10-56-94-92.wav 395 6
mfcc_audio/11-33-06-78-73-29-15.wav.wav 396 6
mfcc_audio/19-27-05-22-36-26-74.wav.wav 397 5
mfcc_audio/14-10-01-94-35-97-39.wav 398 1
mfcc_audio/03-01-01-01-02-02-01.wav.wav 399 1
mfcc_audio/40-12-05-26-58-47-55.wav 400 5
mfcc_audio/94-84-03-55-78-41-96.wav 401 3
mfcc_audio/85-84-03-55-30-22-15.wav 402 3
mfcc_audio/92-94-02-71-21-15-40.wav 403 2
mfcc_audio/84-85-01-20-24-41-19.wav 404 1
mfcc_audio/03-01-01-01-01-01-03.wav.wav 405 1
mfcc_audio/69-87-02-93-78-39-40.wav 406 2
mfcc_audio/03-01-03-02-01-01-24.wav.wav 407 3
mfcc_audio/52-31-01-32-77-11-27.wav 408 1
mfcc_audio/16-69-06-15-61-88-22.wav 409 6
mfcc_audio/30-81-02-58-37-59-98.wav 410 2
mfcc_audio/28-49-03-53-47-58-41.wav.wav 411 3
mfcc_audio/60-76-06-16-38-86-40.wav 412 6
mfcc_audio/26-62-02-41-35-45-92.wav 413 2
mfcc_audio/66-54-03-16-94-73-47.wav 414 3
mfcc_audio/73-97-04-84-36-45-10.wav 415 4
mfcc_audio/64-85-02-39-86-58-70.wav 416 2
mfcc_audio/89-60-03-81-12-11-84.wav 417 3
mfcc_audio

mfcc_audio/27-39-03-16-32-34-81.wav 589 3
mfcc_audio/77-96-04-68-11-24-97.wav 590 4
mfcc_audio/66-63-05-90-54-42-35.wav 591 5
mfcc_audio/25-17-06-47-53-59-34.wav 592 6
mfcc_audio/03-01-05-02-02-02-17.wav.wav 593 5
mfcc_audio/85-30-04-22-26-27-94.wav 594 4
mfcc_audio/64-80-05-14-53-72-20.wav 595 5
mfcc_audio/85-24-04-66-52-97-21.wav.wav 596 4
mfcc_audio/89-63-06-78-68-16-98.wav 597 6
mfcc_audio/59-36-05-74-96-25-28.wav.wav 598 5
mfcc_audio/68-57-04-81-66-30-42.wav.wav 599 4
mfcc_audio/34-16-06-50-97-77-45.wav 600 6
mfcc_audio/36-98-01-22-76-75-31.wav.wav 601 1
mfcc_audio/57-24-01-92-71-97-23.wav 602 1
mfcc_audio/68-81-03-45-22-57-49.wav 603 3
mfcc_audio/43-84-06-36-82-56-75.wav 604 6
mfcc_audio/48-95-04-92-19-42-46.wav.wav 605 4
mfcc_audio/92-94-04-56-55-28-91.wav 606 4
mfcc_audio/38-78-06-88-32-98-81.wav.wav 607 6
mfcc_audio/37-43-04-90-60-64-98.wav 608 4
mfcc_audio/46-77-06-96-80-78-98.wav 609 6
mfcc_audio/70-15-02-89-72-46-60.wav 610 2
mfcc_audio/87-23-02-68-98-75-60.wav 611 2
mfcc_a

mfcc_audio/62-28-04-95-31-12-37.wav 785 4
mfcc_audio/32-84-06-28-97-59-82.wav 786 6
mfcc_audio/70-54-04-19-97-81-15.wav 787 4
mfcc_audio/67-15-05-91-56-72-20.wav 788 5
mfcc_audio/84-11-01-45-93-70-38.wav 789 1
mfcc_audio/14-84-04-45-50-76-15.wav 790 4
mfcc_audio/97-43-05-74-18-22-58.wav 791 5
mfcc_audio/26-22-01-97-56-31-48.wav 792 1
mfcc_audio/19-81-03-43-39-80-88.wav 793 3
mfcc_audio/25-11-02-33-54-29-85.wav 794 2
mfcc_audio/11-97-04-41-13-50-61.wav.wav 795 4
mfcc_audio/46-91-02-53-56-79-71.wav 796 2
mfcc_audio/38-84-06-16-53-27-25.wav.wav 797 6
mfcc_audio/25-12-04-80-66-79-57.wav 798 4
mfcc_audio/74-17-04-89-54-77-32.wav 799 4
mfcc_audio/24-45-04-78-33-52-86.wav 800 4
mfcc_audio/18-42-04-72-16-70-22.wav 801 4
mfcc_audio/18-35-02-40-51-74-26.wav 802 2
mfcc_audio/81-84-04-59-10-64-25.wav 803 4
mfcc_audio/75-27-06-62-17-49-65.wav 804 6
mfcc_audio/61-59-01-63-51-18-70.wav 805 1
mfcc_audio/25-76-06-86-57-63-89.wav 806 6
mfcc_audio/93-65-04-74-50-51-58.wav 807 4
mfcc_audio/78-90-02-88-15-

...

mfcc_audio/31-38-05-33-22-32-79.wav 10197 5
mfcc_audio/86-78-01-26-71-76-90.wav 10198 1
mfcc_audio/90-49-02-20-62-47-22.wav 10199 2
mfcc_audio/14-85-02-16-86-41-20.wav 10200 2
mfcc_audio/65-96-03-95-29-98-33.wav 10201 3
mfcc_audio/52-16-05-70-28-61-75.wav 10202 5
mfcc_audio/70-28-04-67-77-31-14.wav.wav 10203 4
mfcc_audio/71-62-02-43-98-52-33.wav 10204 2
mfcc_audio/29-26-03-98-84-35-69.wav 10205 3
mfcc_audio/38-28-03-13-93-18-32.wav 10206 3
mfcc_audio/31-83-06-11-44-22-36.wav.wav 10207 6
mfcc_audio/03-01-02-01-02-02-11.wav.wav 10208 2
mfcc_audio/56-81-01-76-57-29-94.wav 10209 1
mfcc_audio/63-67-02-96-24-80-49.wav.wav 10210 2
mfcc_audio/13-44-02-37-59-74-87.wav 10211 2
mfcc_audio/96-32-04-59-34-63-30.wav 10212 4
mfcc_audio/41-51-06-42-12-18-10.wav.wav 10213 6
mfcc_audio/56-72-02-67-87-95-88.wav 10214 2
mfcc_audio/54-30-01-60-93-88-39.wav 10215 1
mfcc_audio/36-92-04-35-45-11-18.wav.wav 10216 4
mfcc_audio/74-13-02-34-87-81-84.wav 10217 2
mfcc_audio/73-76-03-82-75-17-48.wav 10218 3
mfcc_aud

mfcc_audio/26-97-06-93-64-83-51.wav 10383 6
mfcc_audio/50-57-01-40-64-11-55.wav 10384 1
mfcc_audio/75-22-03-93-32-52-74.wav 10385 3
mfcc_audio/47-19-04-95-78-62-39.wav 10386 4
mfcc_audio/95-82-01-63-31-21-38.wav 10387 1
mfcc_audio/39-12-05-60-73-24-96.wav.wav 10388 5
mfcc_audio/24-42-03-60-33-87-25.wav 10389 3
mfcc_audio/86-56-06-57-97-10-13.wav 10390 6
mfcc_audio/03-01-05-02-01-01-20.wav.wav 10391 5
mfcc_audio/75-87-02-72-27-30-40.wav.wav 10392 2
mfcc_audio/28-74-03-67-10-57-92.wav 10393 3
mfcc_audio/25-89-03-49-63-74-18.wav 10394 3
mfcc_audio/48-38-03-82-52-49-17.wav 10395 3
mfcc_audio/37-75-05-80-14-59-31.wav 10396 5
mfcc_audio/30-66-03-19-32-24-88.wav 10397 3
mfcc_audio/48-17-06-82-89-88-76.wav 10398 6
mfcc_audio/13-75-02-26-31-90-19.wav.wav 10399 2
mfcc_audio/16-25-04-86-89-76-45.wav 10400 4
mfcc_audio/81-53-03-35-56-62-78.wav.wav 10401 3
mfcc_audio/82-23-06-86-46-72-48.wav.wav 10402 6
mfcc_audio/42-12-03-27-74-17-21.wav 10403 3
mfcc_audio/44-19-04-69-17-36-42.wav 10404 4
mfcc_aud

mfcc_audio/89-19-04-51-43-34-39.wav 10566 4
mfcc_audio/39-13-04-66-12-24-54.wav 10567 4
mfcc_audio/83-81-01-57-92-52-80.wav 10568 1
mfcc_audio/37-15-03-78-16-55-80.wav 10569 3
mfcc_audio/45-55-06-68-49-12-62.wav 10570 6
mfcc_audio/03-01-04-02-01-02-05.wav.wav 10571 4
mfcc_audio/15-26-04-27-64-59-87.wav 10572 4
mfcc_audio/49-98-01-47-91-44-62.wav 10573 1
mfcc_audio/39-14-04-81-55-89-65.wav 10574 4
mfcc_audio/13-61-05-68-18-69-37.wav 10575 5
mfcc_audio/42-26-01-79-10-80-56.wav 10576 1
mfcc_audio/03-01-05-02-02-01-08.wav.wav 10577 5
mfcc_audio/65-70-05-49-38-67-28.wav.wav 10578 5
mfcc_audio/63-67-05-48-35-70-33.wav.wav 10579 5
mfcc_audio/31-15-06-26-19-49-84.wav 10580 6
mfcc_audio/23-70-01-27-91-42-79.wav 10581 1
mfcc_audio/60-70-04-16-59-54-15.wav.wav 10582 4
mfcc_audio/03-01-05-01-02-02-18.wav.wav 10583 5
mfcc_audio/85-50-06-75-48-34-70.wav 10584 6
mfcc_audio/14-28-04-23-32-94-79.wav 10585 4
mfcc_audio/11-53-03-91-13-95-46.wav 10586 3
mfcc_audio/67-16-03-69-59-84-48.wav 10587 3
mfcc_aud

mfcc_audio/25-81-06-61-59-40-98.wav 10751 6
mfcc_audio/48-35-06-14-46-62-91.wav 10752 6
mfcc_audio/36-15-04-91-75-43-11.wav 10753 4
mfcc_audio/29-20-03-30-47-57-11.wav 10754 3
mfcc_audio/29-40-02-33-12-71-41.wav 10755 2
mfcc_audio/62-70-01-28-88-20-56.wav 10756 1
mfcc_audio/31-81-03-50-43-66-86.wav.wav 10757 3
mfcc_audio/10-47-05-30-59-65-51.wav 10758 5
mfcc_audio/66-59-05-64-56-21-22.wav 10759 5
mfcc_audio/78-32-05-81-44-18-52.wav 10760 5
mfcc_audio/52-83-03-63-71-35-45.wav 10761 3
mfcc_audio/62-79-02-33-38-94-93.wav 10762 2
mfcc_audio/67-54-03-21-72-16-36.wav 10763 3
mfcc_audio/88-48-03-14-96-27-66.wav 10764 3
mfcc_audio/38-98-04-74-48-28-88.wav.wav 10765 4
mfcc_audio/33-56-04-28-71-64-40.wav 10766 4
mfcc_audio/89-39-03-77-33-63-14.wav 10767 3
mfcc_audio/47-31-05-46-33-58-44.wav 10768 5
mfcc_audio/57-34-04-50-82-47-35.wav 10769 4
mfcc_audio/31-82-05-35-77-69-20.wav.wav 10770 5
mfcc_audio/46-12-02-42-81-11-76.wav 10771 2
mfcc_audio/56-35-02-97-98-91-20.wav 10772 2
mfcc_audio/84-65-01-

mfcc_audio/43-17-03-78-15-62-87.wav.wav 10941 3
mfcc_audio/03-01-05-01-02-01-16.wav.wav 10942 5
mfcc_audio/50-24-02-83-34-22-29.wav 10943 2
mfcc_audio/18-88-04-80-51-63-82.wav 10944 4
mfcc_audio/35-66-03-80-93-74-51.wav 10945 3
mfcc_audio/88-14-03-97-32-44-70.wav 10946 3
mfcc_audio/51-36-05-28-87-76-20.wav 10947 5
mfcc_audio/46-71-05-57-48-91-68.wav 10948 5
mfcc_audio/91-11-05-69-12-62-52.wav 10949 5
mfcc_audio/74-16-02-48-61-11-34.wav 10950 2
mfcc_audio/61-27-06-87-50-25-43.wav.wav 10951 6
mfcc_audio/52-96-04-20-60-29-64.wav 10952 4
mfcc_audio/31-54-03-89-72-20-81.wav 10953 3
mfcc_audio/44-83-03-18-31-62-42.wav.wav 10954 3
mfcc_audio/20-27-02-26-75-22-58.wav 10955 2
mfcc_audio/93-20-04-35-75-90-43.wav 10956 4
mfcc_audio/46-70-03-87-49-84-95.wav.wav 10957 3
mfcc_audio/71-10-02-14-40-67-79.wav 10958 2
mfcc_audio/21-68-03-28-19-61-16.wav 10959 3
mfcc_audio/77-32-05-19-75-20-76.wav 10960 5
mfcc_audio/34-45-02-17-54-33-63.wav 10961 2
mfcc_audio/81-48-06-89-42-34-22.wav.wav 10962 6
mfcc_aud

In [38]:
len(lst)

10974

In [39]:
# Creating X and y: zip makes a list of all the first elements, and a list of all the second elements.
X, y = zip(*lst)
import numpy as np
X = np.asarray(X)
y = np.asarray(y)
X.shape, y.shape

((10974, 40), (10974,))

In [40]:
X_name = 'X_mfcc_train_par.joblib'
y_name = 'y_mfcc_train_par.joblib'
save_dir = 'mel_gd_joblibs'

savedX = joblib.dump(X, os.path.join(save_dir, X_name))
savedy = joblib.dump(y, os.path.join(save_dir, y_name))

In [41]:
y

array([4, 3, 4, ..., 4, 4, 5])

In [42]:
import time
import os
lst = []

start_time = time.time()

for lab, audio in enumerate(mfcc_df_test.path):
    try:
        #Load librosa array, obtain mfcss, store the file and the mcss information in a new array
        X, sample_rate = librosa.load(audio, res_type='kaiser_fast')
        mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T,axis=0) 
        # The instruction below converts the labels (from 1 to 8) to a series from 0 to 7
        # This is because our predictor needs to start from 0 otherwise it will try to predict also 0.
        file = int(mfcc_df_test.label[lab]) - 1 
        arr = mfccs, file
        lst.append(arr)
        print(audio, lab, mfcc_df_test.label[lab])
        # If the file is not valid, skip it
    except ValueError:
        continue

print("--- Data loaded. Loading time: %s seconds ---" % (time.time() - start_time))

mfcc_audio/41-63-03-54-56-92-81.wav 0 3
mfcc_audio/18-87-05-12-50-20-75.wav 1 5
mfcc_audio/83-88-02-87-35-27-62.wav 2 2
mfcc_audio/56-93-06-84-72-91-69.wav 3 6
mfcc_audio/25-64-03-28-16-59-11.wav 4 3
mfcc_audio/22-79-04-18-63-82-10.wav 5 4
mfcc_audio/11-42-05-31-78-81-67.wav 6 5
mfcc_audio/91-98-01-16-29-27-22.wav.wav 7 1
mfcc_audio/36-96-03-24-79-52-53.wav 8 3
mfcc_audio/87-29-01-93-48-69-91.wav 9 1
mfcc_audio/03-01-02-01-01-01-12.wav.wav 10 2
mfcc_audio/30-96-01-86-63-20-10.wav.wav 11 1
mfcc_audio/29-52-06-56-79-45-33.wav.wav 12 6
mfcc_audio/29-91-06-69-55-46-38.wav 13 6
mfcc_audio/61-72-04-22-35-54-90.wav 14 4
mfcc_audio/88-68-02-48-28-73-15.wav 15 2
mfcc_audio/03-01-02-01-02-01-18.wav.wav 16 2
mfcc_audio/22-35-05-45-90-83-79.wav 17 5
mfcc_audio/15-58-01-33-94-46-79.wav 18 1
mfcc_audio/74-55-05-87-22-53-39.wav 19 5
mfcc_audio/72-36-05-89-22-94-26.wav 20 5
mfcc_audio/61-46-04-76-72-52-14.wav.wav 21 4
mfcc_audio/20-86-02-39-68-29-96.wav 22 2
mfcc_audio/03-01-05-01-01-02-09.wav.wav 23 

mfcc_audio/20-46-05-56-41-50-69.wav 194 5
mfcc_audio/21-49-04-73-88-35-96.wav 195 4
mfcc_audio/47-12-03-34-11-96-77.wav 196 3
mfcc_audio/44-19-06-23-56-65-15.wav 197 6
mfcc_audio/53-11-05-39-15-29-74.wav.wav 198 5
mfcc_audio/84-55-05-28-78-71-72.wav 199 5
mfcc_audio/41-52-04-34-29-28-48.wav 200 4
mfcc_audio/58-77-02-53-80-52-17.wav.wav 201 2
mfcc_audio/38-66-03-89-33-64-88.wav 202 3
mfcc_audio/47-53-05-57-56-62-33.wav.wav 203 5
mfcc_audio/03-01-03-01-02-01-14.wav.wav 204 3
mfcc_audio/66-48-06-31-51-28-17.wav 205 6
mfcc_audio/13-48-03-45-69-24-80.wav 206 3
mfcc_audio/46-95-05-17-86-34-61.wav 207 5
mfcc_audio/72-24-06-33-16-39-56.wav 208 6
mfcc_audio/03-01-03-01-01-02-14.wav.wav 209 3
mfcc_audio/32-36-02-96-86-63-50.wav 210 2
mfcc_audio/52-32-06-97-82-45-62.wav 211 6
mfcc_audio/17-44-06-67-25-60-90.wav 212 6
mfcc_audio/03-01-02-02-02-01-10.wav.wav 213 2
mfcc_audio/98-28-03-33-63-40-13.wav 214 3
mfcc_audio/43-13-02-26-82-27-46.wav 215 2
mfcc_audio/50-78-01-85-38-48-14.wav 216 1
mfcc_audio

mfcc_audio/20-93-03-92-54-33-56.wav 393 3
mfcc_audio/43-98-02-38-54-33-46.wav 394 2
mfcc_audio/16-69-03-20-57-91-11.wav 395 3
mfcc_audio/63-16-04-45-31-34-24.wav 396 4
mfcc_audio/35-82-03-46-10-87-94.wav 397 3
mfcc_audio/53-20-04-79-67-98-94.wav 398 4
mfcc_audio/57-86-03-54-20-85-94.wav 399 3
mfcc_audio/41-35-03-61-47-68-67.wav.wav 400 3
mfcc_audio/80-12-05-70-96-73-89.wav 401 5
mfcc_audio/64-91-01-95-92-41-96.wav 402 1
mfcc_audio/57-63-02-31-73-36-61.wav 403 2
mfcc_audio/58-16-03-50-81-24-14.wav.wav 404 3
mfcc_audio/59-39-02-76-55-52-80.wav 405 2
mfcc_audio/75-22-01-89-80-77-45.wav 406 1
mfcc_audio/18-93-02-51-66-27-64.wav 407 2
mfcc_audio/55-39-06-11-46-38-50.wav 408 6
mfcc_audio/31-78-02-30-82-46-97.wav.wav 409 2
mfcc_audio/76-10-06-23-46-39-89.wav 410 6
mfcc_audio/90-56-05-19-68-39-65.wav 411 5
mfcc_audio/89-77-03-54-57-90-37.wav 412 3
mfcc_audio/84-56-03-75-23-55-33.wav 413 3
mfcc_audio/71-21-01-72-45-23-68.wav.wav 414 1
mfcc_audio/60-39-02-38-84-21-89.wav 415 2
mfcc_audio/70-15-0

mfcc_audio/37-40-02-58-41-17-21.wav 587 2
mfcc_audio/52-38-01-78-15-59-57.wav 588 1
mfcc_audio/31-14-01-56-96-53-61.wav 589 1
mfcc_audio/11-72-04-98-48-34-22.wav 590 4
mfcc_audio/11-86-05-72-49-77-51.wav 591 5
mfcc_audio/69-61-01-14-70-75-97.wav 592 1
mfcc_audio/66-27-02-53-65-92-70.wav 593 2
mfcc_audio/11-85-01-65-98-72-60.wav.wav 594 1
mfcc_audio/28-46-05-51-41-78-49.wav 595 5
mfcc_audio/11-63-01-90-17-70-44.wav 596 1
mfcc_audio/95-45-06-74-22-41-88.wav 597 6
mfcc_audio/64-89-01-28-12-97-81.wav 598 1
mfcc_audio/73-75-01-46-53-90-22.wav 599 1
mfcc_audio/45-58-06-25-14-51-71.wav 600 6
mfcc_audio/54-13-05-81-26-66-22.wav.wav 601 5
mfcc_audio/03-01-06-01-01-01-21.wav.wav 602 6
mfcc_audio/03-01-06-01-02-02-09.wav.wav 603 6
mfcc_audio/42-78-05-65-44-82-48.wav.wav 604 5
mfcc_audio/15-40-04-45-55-30-94.wav 605 4
mfcc_audio/73-55-03-77-92-53-52.wav 606 3
mfcc_audio/50-87-02-16-94-13-68.wav.wav 607 2
mfcc_audio/22-90-05-71-83-11-36.wav 608 5
mfcc_audio/76-25-05-87-79-32-34.wav 609 5
mfcc_audio

mfcc_audio/59-86-01-81-60-49-44.wav.wav 786 1
mfcc_audio/81-92-03-64-39-49-24.wav 787 3
mfcc_audio/47-89-05-17-19-46-32.wav.wav 788 5
mfcc_audio/88-41-02-28-51-37-93.wav 789 2
mfcc_audio/62-56-02-23-43-45-90.wav.wav 790 2
mfcc_audio/77-14-04-69-70-56-27.wav 791 4
mfcc_audio/68-40-05-79-93-64-66.wav 792 5
mfcc_audio/89-39-05-69-15-32-24.wav.wav 793 5
mfcc_audio/30-44-03-54-84-94-55.wav 794 3
mfcc_audio/71-77-05-35-38-79-74.wav 795 5
mfcc_audio/30-71-05-19-55-87-20.wav 796 5
mfcc_audio/67-24-05-69-58-15-93.wav 797 5
mfcc_audio/87-95-03-71-22-49-54.wav 798 3
mfcc_audio/64-85-05-79-42-98-81.wav 799 5
mfcc_audio/21-48-01-72-51-44-12.wav 800 1
mfcc_audio/20-17-06-91-79-47-84.wav 801 6
mfcc_audio/59-39-05-15-50-53-79.wav.wav 802 5
mfcc_audio/03-01-02-02-02-02-11.wav.wav 803 2
mfcc_audio/34-35-01-68-66-64-39.wav 804 1
mfcc_audio/14-51-02-43-81-33-44.wav.wav 805 2
mfcc_audio/46-63-02-97-28-78-65.wav.wav 806 2
mfcc_audio/10-93-02-65-15-32-58.wav.wav 807 2
mfcc_audio/24-32-04-66-88-59-56.wav 808 

...

mfcc_audio/62-29-02-72-54-23-55.wav 1936 2
mfcc_audio/29-42-04-66-89-92-47.wav 1937 4
mfcc_audio/71-69-05-90-26-33-11.wav.wav 1938 5
mfcc_audio/88-58-02-12-40-62-16.wav.wav 1939 2
mfcc_audio/22-85-04-75-54-47-43.wav 1940 4
mfcc_audio/76-16-06-45-79-69-11.wav 1941 6
mfcc_audio/71-28-01-91-41-64-50.wav 1942 1
mfcc_audio/70-16-04-55-21-23-82.wav 1943 4
mfcc_audio/26-92-05-46-90-49-61.wav.wav 1944 5
mfcc_audio/37-38-04-32-31-94-61.wav 1945 4
mfcc_audio/80-47-01-23-44-16-72.wav 1946 1
mfcc_audio/25-40-06-64-16-50-38.wav 1947 6
mfcc_audio/63-31-02-14-93-67-89.wav.wav 1948 2
mfcc_audio/65-85-01-94-20-32-81.wav.wav 1949 1
mfcc_audio/25-66-03-85-17-32-24.wav 1950 3
mfcc_audio/11-52-01-21-14-50-66.wav 1951 1
mfcc_audio/25-98-02-44-74-80-85.wav 1952 2
mfcc_audio/49-94-06-14-75-69-15.wav.wav 1953 6
mfcc_audio/45-23-02-93-48-52-75.wav 1954 2
mfcc_audio/72-47-02-61-29-82-19.wav 1955 2
mfcc_audio/11-49-04-28-83-53-65.wav 1956 4
mfcc_audio/35-18-01-54-36-22-29.wav.wav 1957 1
mfcc_audio/70-29-02-53-82-

mfcc_audio/19-90-03-88-85-49-24.wav 2126 3
mfcc_audio/95-54-05-43-96-76-13.wav 2127 5
mfcc_audio/89-24-02-32-88-95-41.wav.wav 2128 2
mfcc_audio/11-49-05-89-46-57-51.wav.wav 2129 5
mfcc_audio/75-73-06-56-34-74-63.wav 2130 6
mfcc_audio/83-12-03-37-64-55-68.wav 2131 3
mfcc_audio/94-53-03-52-95-87-12.wav 2132 3
mfcc_audio/55-80-04-41-30-29-81.wav 2133 4
mfcc_audio/37-96-01-97-70-26-40.wav 2134 1
mfcc_audio/42-78-05-26-84-58-31.wav.wav 2135 5
mfcc_audio/65-46-06-50-72-23-52.wav.wav 2136 6
mfcc_audio/37-54-06-19-74-77-73.wav 2137 6
mfcc_audio/03-01-02-01-02-01-06.wav.wav 2138 2
mfcc_audio/81-18-05-20-63-89-34.wav 2139 5
mfcc_audio/54-38-05-77-95-82-43.wav 2140 5
mfcc_audio/55-32-05-36-14-90-12.wav 2141 5
mfcc_audio/22-74-03-10-85-29-58.wav 2142 3
mfcc_audio/85-88-03-17-55-34-67.wav 2143 3
mfcc_audio/13-61-06-60-30-95-96.wav 2144 6
mfcc_audio/44-20-06-32-10-34-78.wav 2145 6
mfcc_audio/89-75-01-51-16-60-73.wav.wav 2146 1
mfcc_audio/76-37-03-21-87-53-19.wav 2147 3
mfcc_audio/51-60-05-72-82-46-6

mfcc_audio/03-01-05-01-01-01-06.wav.wav 2319 5
mfcc_audio/27-13-02-32-70-73-35.wav 2320 2
mfcc_audio/31-41-06-50-88-53-12.wav 2321 6
mfcc_audio/55-15-05-36-76-51-84.wav 2322 5
mfcc_audio/12-11-06-68-72-40-27.wav 2323 6
mfcc_audio/23-79-03-31-56-80-25.wav 2324 3
mfcc_audio/88-46-03-92-76-37-87.wav 2325 3
mfcc_audio/56-98-06-88-65-61-66.wav 2326 6
mfcc_audio/59-24-06-53-78-64-72.wav.wav 2327 6
mfcc_audio/82-51-04-52-50-20-36.wav 2328 4
mfcc_audio/39-69-02-18-73-80-13.wav.wav 2329 2
mfcc_audio/79-34-01-41-96-18-86.wav.wav 2330 1
mfcc_audio/03-01-04-01-01-01-07.wav.wav 2331 4
mfcc_audio/03-01-03-02-01-01-05.wav.wav 2332 3
mfcc_audio/10-96-04-13-34-44-58.wav.wav 2333 4
mfcc_audio/28-91-03-43-45-74-37.wav 2334 3
mfcc_audio/78-57-06-71-96-46-39.wav 2335 6
mfcc_audio/46-26-03-51-73-71-97.wav 2336 3
mfcc_audio/28-81-06-31-30-11-27.wav.wav 2337 6
mfcc_audio/76-11-02-17-82-65-31.wav 2338 2
mfcc_audio/54-33-02-58-36-16-57.wav 2339 2
mfcc_audio/26-34-05-82-79-33-12.wav 2340 5
mfcc_audio/42-94-02-36

mfcc_audio/26-46-05-45-66-51-91.wav 2515 5
mfcc_audio/65-95-05-93-55-83-90.wav 2516 5
mfcc_audio/21-93-02-18-59-76-69.wav 2517 2
mfcc_audio/17-72-05-88-79-19-91.wav 2518 5
mfcc_audio/69-80-06-33-77-41-42.wav 2519 6
mfcc_audio/62-25-04-74-51-83-73.wav 2520 4
mfcc_audio/29-11-05-84-43-85-48.wav 2521 5
mfcc_audio/70-96-06-55-11-60-68.wav 2522 6
mfcc_audio/19-85-03-52-35-87-33.wav 2523 3
mfcc_audio/83-25-06-22-26-17-85.wav 2524 6
mfcc_audio/51-14-01-69-93-70-83.wav 2525 1
mfcc_audio/97-20-05-50-46-70-54.wav 2526 5
mfcc_audio/40-62-01-25-75-67-93.wav 2527 1
mfcc_audio/31-44-04-26-29-27-41.wav 2528 4
mfcc_audio/03-01-02-01-02-02-19.wav.wav 2529 2
mfcc_audio/45-51-05-53-75-24-71.wav 2530 5
mfcc_audio/87-27-03-19-31-57-67.wav 2531 3
mfcc_audio/31-35-06-53-25-98-78.wav 2532 6
mfcc_audio/30-95-06-63-58-53-36.wav 2533 6
mfcc_audio/95-96-01-28-40-42-77.wav 2534 1
mfcc_audio/03-01-03-02-01-01-16.wav.wav 2535 3
mfcc_audio/91-40-04-22-13-73-11.wav.wav 2536 4
mfcc_audio/84-10-05-90-43-92-78.wav 2537 5

mfcc_audio/70-49-01-82-41-45-56.wav.wav 2704 1
mfcc_audio/32-82-03-86-62-31-10.wav 2705 3
mfcc_audio/40-71-03-64-36-24-81.wav 2706 3
mfcc_audio/97-77-04-51-81-78-24.wav 2707 4
mfcc_audio/64-92-05-62-44-59-51.wav 2708 5
mfcc_audio/12-69-06-53-45-48-94.wav 2709 6
mfcc_audio/40-96-01-62-85-59-15.wav 2710 1
mfcc_audio/45-12-03-70-23-65-68.wav 2711 3
mfcc_audio/52-25-03-85-88-37-12.wav 2712 3
mfcc_audio/69-53-05-33-58-95-56.wav 2713 5
mfcc_audio/91-28-05-98-33-17-20.wav 2714 5
mfcc_audio/15-98-04-70-18-42-47.wav 2715 4
mfcc_audio/76-48-03-71-95-19-98.wav.wav 2716 3
mfcc_audio/73-29-04-88-27-91-10.wav.wav 2717 4
mfcc_audio/49-53-02-70-95-10-55.wav 2718 2
mfcc_audio/03-01-04-01-01-01-01.wav.wav 2719 4
mfcc_audio/66-42-03-88-81-84-16.wav 2720 3
mfcc_audio/20-17-02-39-78-86-67.wav.wav 2721 2
mfcc_audio/54-80-01-82-16-60-34.wav 2722 1
mfcc_audio/59-83-02-84-62-24-74.wav 2723 2
mfcc_audio/58-36-06-16-35-78-14.wav 2724 6
mfcc_audio/34-55-02-39-45-35-85.wav 2725 2
mfcc_audio/03-01-05-02-02-01-05.wa

In [43]:
len(lst)

2744

In [44]:
# Creating X and y: zip makes a list of all the first elements, and a list of all the second elements.
X, y = zip(*lst)
import numpy as np
X = np.asarray(X)
y = np.asarray(y)
X.shape, y.shape

((2744, 40), (2744,))

In [45]:
X_name = 'X_mfcc_test_par.joblib'
y_name = 'y_mfcc_test_par.joblib'
save_dir = 'mel_gd_joblibs'

savedX = joblib.dump(X, os.path.join(save_dir, X_name))
savedy = joblib.dump(y, os.path.join(save_dir, y_name))

In [46]:
y

array([2, 4, 1, ..., 3, 3, 3])

In [47]:
mel_train_df

,path,label
0,data_cropped/89-85-05-49-65-24-92.png,5
1,data_cropped/03-01-04-02-01-02-16.png,4
2,data_cropped/82-75-05-47-94-55-26.png,5
3,data_cropped/12-96-05-14-41-31-19.png,5
4,data_cropped/41-27-04-53-12-39-29.png,4
...,...,...
10969,data_cropped/22-47-01-17-54-86-97.png,1
10970,data_cropped/49-35-01-52-48-38-39.png,1
10971,data_cropped/12-71-05-39-73-74-32.png,5
10972,data_cropped/38-23-05-33-79-81-92.png,5


In [48]:
mel_test_df

,path,label
0,data_cropped/41-63-03-54-56-92-81.png,3
1,data_cropped/18-87-05-12-50-20-75.png,5
2,data_cropped/83-88-02-87-35-27-62.png,2
3,data_cropped/56-93-06-84-72-91-69.png,6
4,data_cropped/25-64-03-28-16-59-11.png,3
...,...,...
2739,data_cropped/74-76-04-71-24-15-48.png,4
2740,data_cropped/34-37-02-35-15-10-63.png,2
2741,data_cropped/75-20-04-65-12-28-39.png,4
2742,data_cropped/89-51-04-54-29-96-64.png,4


In [49]:
data = []
label =[]

for lab, img in enumerate(mel_train_df.path):
  try:
    image = load_img(img, target_size=(224, 224))
    # convert the image pixels to a numpy array 
    image = img_to_array(image) 
    # reshape data for the model
    image = image.reshape((image.shape[0], image.shape[1], image.shape[2]))
    # prepare the image for the VGG model
    image = preprocess_input(image)
    data.append(image)
    label.append(int(mel_train_df.label[lab])-1)
    print(lab, img)
  except:
    print("Error loading image", lab)
  

#Converting lists into numpy arrays
data = np.array(data)
label = np.array(label)

print(data.shape, label.shape)

0 data_cropped/89-85-05-49-65-24-92.png
1 data_cropped/03-01-04-02-01-02-16.png
2 data_cropped/82-75-05-47-94-55-26.png
3 data_cropped/12-96-05-14-41-31-19.png
4 data_cropped/41-27-04-53-12-39-29.png
5 data_cropped/29-38-04-70-77-19-39.png
6 data_cropped/84-33-01-75-86-23-70.png
7 data_cropped/49-66-03-68-98-57-52.png
8 data_cropped/87-56-05-89-10-59-16.png
9 data_cropped/69-14-03-37-18-95-79.png
10 data_cropped/57-24-05-69-29-90-66.png
11 data_cropped/95-39-05-17-93-69-68.png
12 data_cropped/65-78-06-54-20-81-91.png
13 data_cropped/35-10-04-36-62-58-91.png
14 data_cropped/59-28-02-97-26-93-35.png
15 data_cropped/50-57-04-84-59-82-81.png
16 data_cropped/85-41-05-11-17-24-22.png
17 data_cropped/40-67-06-64-82-69-44.png
18 data_cropped/71-43-06-79-36-45-44.png
19 data_cropped/17-90-01-79-81-88-20.png
20 data_cropped/25-17-06-73-39-19-18.png
21 data_cropped/67-58-06-17-64-10-76.png
22 data_cropped/88-64-02-48-43-90-16.png
23 data_cropped/12-22-03-82-26-83-24.png
24 data_cropped/33-25-05-3

205 data_cropped/51-94-03-71-45-55-44.png
206 data_cropped/80-84-05-59-18-57-22.png
207 data_cropped/53-45-04-90-89-66-96.png
208 data_cropped/75-51-05-24-60-40-58.png
209 data_cropped/03-01-02-01-02-01-17.png
210 data_cropped/37-54-02-36-27-82-94.png
211 data_cropped/24-39-05-11-90-67-98.png
212 data_cropped/50-62-02-66-63-70-24.png
213 data_cropped/28-18-05-13-89-62-73.png
214 data_cropped/39-83-04-40-69-37-47.png
215 data_cropped/34-58-04-93-63-11-52.png
216 data_cropped/54-50-04-44-43-83-34.png
217 data_cropped/35-22-04-27-64-93-42.png
218 data_cropped/16-18-01-74-66-23-84.png
219 data_cropped/30-49-01-91-69-73-89.png
220 data_cropped/76-32-06-64-50-22-17.png
221 data_cropped/69-78-04-20-25-46-42.png
222 data_cropped/03-01-01-01-02-01-14.png
223 data_cropped/37-64-05-67-93-94-66.png
224 data_cropped/03-01-04-02-01-01-16.png
225 data_cropped/79-43-06-73-50-38-65.png
226 data_cropped/44-15-05-16-55-14-81.png
227 data_cropped/92-72-05-36-87-20-79.png
228 data_cropped/27-48-06-13-42-72

409 data_cropped/16-69-06-15-61-88-22.png
410 data_cropped/30-81-02-58-37-59-98.png
411 data_cropped/28-49-03-53-47-58-41.png
412 data_cropped/60-76-06-16-38-86-40.png
413 data_cropped/26-62-02-41-35-45-92.png
414 data_cropped/66-54-03-16-94-73-47.png
415 data_cropped/73-97-04-84-36-45-10.png
416 data_cropped/64-85-02-39-86-58-70.png
417 data_cropped/89-60-03-81-12-11-84.png
418 data_cropped/77-66-03-25-92-46-81.png
419 data_cropped/38-54-04-21-67-79-83.png
420 data_cropped/63-47-06-22-70-37-28.png
421 data_cropped/71-33-06-60-13-94-23.png
422 data_cropped/34-43-05-31-72-26-28.png
423 data_cropped/32-34-04-63-93-89-57.png
424 data_cropped/67-92-04-87-59-71-52.png
425 data_cropped/13-33-06-72-16-88-36.png
426 data_cropped/45-35-04-27-33-92-26.png
427 data_cropped/29-89-01-30-67-22-38.png
428 data_cropped/55-90-01-20-86-64-41.png
429 data_cropped/66-46-05-77-34-27-75.png
430 data_cropped/50-47-06-90-14-89-31.png
431 data_cropped/28-40-04-14-47-44-88.png
432 data_cropped/82-52-06-48-25-91

612 data_cropped/03-01-01-01-01-02-22.png
613 data_cropped/71-69-05-27-67-79-19.png
614 data_cropped/32-22-04-41-34-94-16.png
615 data_cropped/11-96-03-68-43-72-49.png
616 data_cropped/67-29-04-50-75-57-79.png
617 data_cropped/65-31-03-40-72-24-83.png
618 data_cropped/43-36-04-17-79-76-20.png
619 data_cropped/36-70-01-10-96-56-17.png
620 data_cropped/29-94-01-21-45-60-10.png
621 data_cropped/37-65-02-22-51-98-70.png
622 data_cropped/27-49-06-82-97-94-25.png
623 data_cropped/65-95-01-12-91-23-93.png
624 data_cropped/36-40-01-35-79-61-97.png
625 data_cropped/76-92-05-71-96-87-89.png
626 data_cropped/21-97-02-32-36-41-93.png
627 data_cropped/23-33-05-92-73-94-88.png
628 data_cropped/61-27-06-10-70-42-65.png
629 data_cropped/03-01-05-01-02-01-09.png
630 data_cropped/76-11-05-21-79-87-13.png
631 data_cropped/46-28-01-35-13-26-10.png
632 data_cropped/26-38-05-93-33-84-65.png
633 data_cropped/30-21-06-55-22-93-68.png
634 data_cropped/95-68-02-92-65-59-36.png
635 data_cropped/92-78-04-11-40-53

809 data_cropped/86-65-01-97-12-34-18.png
810 data_cropped/82-90-05-77-67-78-58.png
811 data_cropped/70-26-02-45-95-92-89.png
812 data_cropped/15-57-06-48-28-18-43.png
813 data_cropped/13-44-03-22-72-46-56.png
814 data_cropped/37-88-06-97-36-33-70.png
815 data_cropped/79-47-03-67-42-26-11.png
816 data_cropped/56-18-05-84-50-79-48.png
817 data_cropped/57-22-02-37-23-91-70.png
818 data_cropped/81-29-02-35-13-34-21.png
819 data_cropped/40-37-01-67-36-76-51.png
820 data_cropped/87-82-01-42-61-67-38.png
821 data_cropped/24-59-01-56-93-65-32.png
822 data_cropped/47-38-06-13-22-84-67.png
823 data_cropped/03-01-04-01-01-02-11.png
824 data_cropped/03-01-04-01-01-01-14.png
825 data_cropped/58-72-05-71-46-73-79.png
826 data_cropped/81-52-03-97-72-37-10.png
827 data_cropped/93-59-03-78-87-90-94.png
828 data_cropped/03-01-06-01-01-02-15.png
829 data_cropped/29-81-06-21-79-59-83.png
830 data_cropped/82-26-04-30-76-72-61.png
831 data_cropped/55-82-03-35-96-88-70.png
832 data_cropped/35-77-03-78-30-45

...

10201 data_cropped/65-96-03-95-29-98-33.png
10202 data_cropped/52-16-05-70-28-61-75.png
10203 data_cropped/70-28-04-67-77-31-14.png
10204 data_cropped/71-62-02-43-98-52-33.png
10205 data_cropped/29-26-03-98-84-35-69.png
10206 data_cropped/38-28-03-13-93-18-32.png
10207 data_cropped/31-83-06-11-44-22-36.png
10208 data_cropped/03-01-02-01-02-02-11.png
10209 data_cropped/56-81-01-76-57-29-94.png
10210 data_cropped/63-67-02-96-24-80-49.png
10211 data_cropped/13-44-02-37-59-74-87.png
10212 data_cropped/96-32-04-59-34-63-30.png
10213 data_cropped/41-51-06-42-12-18-10.png
10214 data_cropped/56-72-02-67-87-95-88.png
10215 data_cropped/54-30-01-60-93-88-39.png
10216 data_cropped/36-92-04-35-45-11-18.png
10217 data_cropped/74-13-02-34-87-81-84.png
10218 data_cropped/73-76-03-82-75-17-48.png
10219 data_cropped/40-64-02-25-92-46-19.png
10220 data_cropped/27-38-06-47-13-34-91.png
10221 data_cropped/53-35-03-29-93-43-55.png
10222 data_cropped/11-70-01-14-50-65-53.png
10223 data_cropped/70-14-03-82-7

10388 data_cropped/39-12-05-60-73-24-96.png
10389 data_cropped/24-42-03-60-33-87-25.png
10390 data_cropped/86-56-06-57-97-10-13.png
10391 data_cropped/03-01-05-02-01-01-20.png
10392 data_cropped/75-87-02-72-27-30-40.png
10393 data_cropped/28-74-03-67-10-57-92.png
10394 data_cropped/25-89-03-49-63-74-18.png
10395 data_cropped/48-38-03-82-52-49-17.png
10396 data_cropped/37-75-05-80-14-59-31.png
10397 data_cropped/30-66-03-19-32-24-88.png
10398 data_cropped/48-17-06-82-89-88-76.png
10399 data_cropped/13-75-02-26-31-90-19.png
10400 data_cropped/16-25-04-86-89-76-45.png
10401 data_cropped/81-53-03-35-56-62-78.png
10402 data_cropped/82-23-06-86-46-72-48.png
10403 data_cropped/42-12-03-27-74-17-21.png
10404 data_cropped/44-19-04-69-17-36-42.png
10405 data_cropped/11-10-06-52-22-36-74.png
10406 data_cropped/57-51-03-91-11-23-33.png
10407 data_cropped/42-64-03-93-41-79-92.png
10408 data_cropped/03-01-06-01-02-01-07.png
10409 data_cropped/50-92-01-37-79-63-86.png
10410 data_cropped/27-15-03-17-7

10582 data_cropped/60-70-04-16-59-54-15.png
10583 data_cropped/03-01-05-01-02-02-18.png
10584 data_cropped/85-50-06-75-48-34-70.png
10585 data_cropped/14-28-04-23-32-94-79.png
10586 data_cropped/11-53-03-91-13-95-46.png
10587 data_cropped/67-16-03-69-59-84-48.png
10588 data_cropped/03-01-04-02-02-01-08.png
10589 data_cropped/25-30-05-34-76-35-79.png
10590 data_cropped/97-49-04-80-76-55-23.png
10591 data_cropped/77-35-06-66-24-87-86.png
10592 data_cropped/85-15-02-61-53-87-56.png
10593 data_cropped/29-18-06-15-28-36-30.png
10594 data_cropped/57-69-04-47-74-33-46.png
10595 data_cropped/40-50-02-59-88-78-89.png
10596 data_cropped/38-48-05-79-35-84-63.png
10597 data_cropped/68-26-04-80-38-31-57.png
10598 data_cropped/39-88-06-40-36-47-74.png
10599 data_cropped/93-44-06-24-53-65-47.png
10600 data_cropped/62-75-03-19-97-87-30.png
10601 data_cropped/20-43-05-61-80-69-98.png
10602 data_cropped/39-43-05-84-85-75-18.png
10603 data_cropped/31-53-01-33-28-44-80.png
10604 data_cropped/19-74-06-85-3

10779 data_cropped/36-74-01-33-12-38-28.png
10780 data_cropped/21-83-02-49-90-85-57.png
10781 data_cropped/96-50-01-27-61-60-71.png
10782 data_cropped/44-90-05-43-45-27-25.png
10783 data_cropped/74-29-04-86-93-50-56.png
10784 data_cropped/63-83-02-12-61-52-82.png
10785 data_cropped/23-61-03-84-25-69-72.png
10786 data_cropped/16-45-06-44-55-18-27.png
10787 data_cropped/95-80-05-71-74-15-53.png
10788 data_cropped/57-20-06-28-29-60-73.png
10789 data_cropped/64-87-02-36-58-34-19.png
10790 data_cropped/70-15-02-60-95-98-94.png
10791 data_cropped/85-31-04-50-32-76-19.png
10792 data_cropped/26-89-04-64-51-35-91.png
10793 data_cropped/43-36-04-69-66-55-62.png
10794 data_cropped/47-37-03-64-56-19-17.png
10795 data_cropped/24-28-01-91-16-33-14.png
10796 data_cropped/10-55-03-84-95-43-35.png
10797 data_cropped/27-49-06-61-57-55-35.png
10798 data_cropped/48-78-05-41-20-18-64.png
10799 data_cropped/66-94-04-49-96-98-65.png
10800 data_cropped/17-63-01-26-32-73-27.png
10801 data_cropped/23-66-03-21-7

(10974, 224, 224, 3) (10974,)


In [50]:
X_name = 'X_mel_in_mfcc_train_par.joblib'
y_name = 'y_mel_in_mfcc_train_par.joblib'
save_dir = 'mel_gd_joblibs'

savedX = joblib.dump(data, os.path.join(save_dir, X_name))
savedy = joblib.dump(label, os.path.join(save_dir, y_name))

In [51]:
data = []
label =[]

for lab, img in enumerate(mel_test_df.path):
  try:
    image = load_img(img, target_size=(224, 224))
    # convert the image pixels to a numpy array 
    image = img_to_array(image) 
    # reshape data for the model
    image = image.reshape((image.shape[0], image.shape[1], image.shape[2]))
    # prepare the image for the VGG model
    image = preprocess_input(image)
    data.append(image)
    label.append(int(mel_test_df.label[lab])-1)
    print(lab, img)
  except:
    print("Error loading image", lab)
  

#Converting lists into numpy arrays
data = np.array(data)
label = np.array(label)

print(data.shape, label.shape)

0 data_cropped/41-63-03-54-56-92-81.png
1 data_cropped/18-87-05-12-50-20-75.png
2 data_cropped/83-88-02-87-35-27-62.png
3 data_cropped/56-93-06-84-72-91-69.png
4 data_cropped/25-64-03-28-16-59-11.png
5 data_cropped/22-79-04-18-63-82-10.png
6 data_cropped/11-42-05-31-78-81-67.png
7 data_cropped/91-98-01-16-29-27-22.png
8 data_cropped/36-96-03-24-79-52-53.png
9 data_cropped/87-29-01-93-48-69-91.png
10 data_cropped/03-01-02-01-01-01-12.png
11 data_cropped/30-96-01-86-63-20-10.png
12 data_cropped/29-52-06-56-79-45-33.png
13 data_cropped/29-91-06-69-55-46-38.png
14 data_cropped/61-72-04-22-35-54-90.png
15 data_cropped/88-68-02-48-28-73-15.png
16 data_cropped/03-01-02-01-02-01-18.png
17 data_cropped/22-35-05-45-90-83-79.png
18 data_cropped/15-58-01-33-94-46-79.png
19 data_cropped/74-55-05-87-22-53-39.png
20 data_cropped/72-36-05-89-22-94-26.png
21 data_cropped/61-46-04-76-72-52-14.png
22 data_cropped/20-86-02-39-68-29-96.png
23 data_cropped/03-01-05-01-01-02-09.png
24 data_cropped/46-74-02-4

200 data_cropped/41-52-04-34-29-28-48.png
201 data_cropped/58-77-02-53-80-52-17.png
202 data_cropped/38-66-03-89-33-64-88.png
203 data_cropped/47-53-05-57-56-62-33.png
204 data_cropped/03-01-03-01-02-01-14.png
205 data_cropped/66-48-06-31-51-28-17.png
206 data_cropped/13-48-03-45-69-24-80.png
207 data_cropped/46-95-05-17-86-34-61.png
208 data_cropped/72-24-06-33-16-39-56.png
209 data_cropped/03-01-03-01-01-02-14.png
210 data_cropped/32-36-02-96-86-63-50.png
211 data_cropped/52-32-06-97-82-45-62.png
212 data_cropped/17-44-06-67-25-60-90.png
213 data_cropped/03-01-02-02-02-01-10.png
214 data_cropped/98-28-03-33-63-40-13.png
215 data_cropped/43-13-02-26-82-27-46.png
216 data_cropped/50-78-01-85-38-48-14.png
217 data_cropped/03-01-05-02-02-02-03.png
218 data_cropped/44-17-03-67-66-22-26.png
219 data_cropped/21-34-05-73-87-57-75.png
220 data_cropped/95-80-01-44-56-26-70.png
221 data_cropped/64-92-05-95-51-19-75.png
222 data_cropped/35-60-06-23-84-66-72.png
223 data_cropped/97-27-04-39-90-89

403 data_cropped/57-63-02-31-73-36-61.png
404 data_cropped/58-16-03-50-81-24-14.png
405 data_cropped/59-39-02-76-55-52-80.png
406 data_cropped/75-22-01-89-80-77-45.png
407 data_cropped/18-93-02-51-66-27-64.png
408 data_cropped/55-39-06-11-46-38-50.png
409 data_cropped/31-78-02-30-82-46-97.png
410 data_cropped/76-10-06-23-46-39-89.png
411 data_cropped/90-56-05-19-68-39-65.png
412 data_cropped/89-77-03-54-57-90-37.png
413 data_cropped/84-56-03-75-23-55-33.png
414 data_cropped/71-21-01-72-45-23-68.png
415 data_cropped/60-39-02-38-84-21-89.png
416 data_cropped/70-15-03-92-76-57-33.png
417 data_cropped/65-19-04-64-29-61-46.png
418 data_cropped/15-40-02-29-55-62-25.png
419 data_cropped/51-90-04-34-45-65-53.png
420 data_cropped/74-18-04-48-24-96-82.png
421 data_cropped/22-60-01-90-12-31-40.png
422 data_cropped/17-33-02-19-84-64-82.png
423 data_cropped/88-32-04-98-27-89-94.png
424 data_cropped/44-27-01-21-78-33-49.png
425 data_cropped/03-01-05-01-02-01-21.png
426 data_cropped/16-17-06-61-72-35

600 data_cropped/45-58-06-25-14-51-71.png
601 data_cropped/54-13-05-81-26-66-22.png
602 data_cropped/03-01-06-01-01-01-21.png
603 data_cropped/03-01-06-01-02-02-09.png
604 data_cropped/42-78-05-65-44-82-48.png
605 data_cropped/15-40-04-45-55-30-94.png
606 data_cropped/73-55-03-77-92-53-52.png
607 data_cropped/50-87-02-16-94-13-68.png
608 data_cropped/22-90-05-71-83-11-36.png
609 data_cropped/76-25-05-87-79-32-34.png
610 data_cropped/54-27-02-92-64-94-86.png
611 data_cropped/87-15-05-94-23-60-83.png
612 data_cropped/28-95-06-73-53-30-76.png
613 data_cropped/45-87-06-14-39-23-16.png
614 data_cropped/95-51-06-65-23-97-40.png
615 data_cropped/18-44-06-73-76-95-89.png
616 data_cropped/43-17-05-33-27-55-12.png
617 data_cropped/03-01-01-01-02-02-10.png
618 data_cropped/64-45-04-62-28-32-35.png
619 data_cropped/82-55-05-92-67-23-64.png
620 data_cropped/55-61-05-53-13-65-66.png
621 data_cropped/25-35-06-78-89-48-76.png
622 data_cropped/65-58-06-34-47-48-45.png
623 data_cropped/52-15-03-27-67-94

798 data_cropped/87-95-03-71-22-49-54.png
799 data_cropped/64-85-05-79-42-98-81.png
800 data_cropped/21-48-01-72-51-44-12.png
801 data_cropped/20-17-06-91-79-47-84.png
802 data_cropped/59-39-05-15-50-53-79.png
803 data_cropped/03-01-02-02-02-02-11.png
804 data_cropped/34-35-01-68-66-64-39.png
805 data_cropped/14-51-02-43-81-33-44.png
806 data_cropped/46-63-02-97-28-78-65.png
807 data_cropped/10-93-02-65-15-32-58.png
808 data_cropped/24-32-04-66-88-59-56.png
809 data_cropped/39-27-04-72-43-20-34.png
810 data_cropped/13-33-06-27-84-82-85.png
811 data_cropped/19-89-04-40-47-78-56.png
812 data_cropped/61-74-05-55-66-40-32.png
813 data_cropped/13-40-01-49-90-95-72.png
814 data_cropped/46-65-05-73-21-13-97.png
815 data_cropped/77-81-04-41-22-62-98.png
816 data_cropped/38-22-02-50-70-87-44.png
817 data_cropped/12-63-04-20-69-82-44.png
818 data_cropped/42-46-05-29-36-55-27.png
819 data_cropped/74-49-04-94-24-67-12.png
820 data_cropped/72-50-04-76-69-63-31.png
821 data_cropped/39-41-06-46-25-55

...

1780 data_cropped/71-20-01-22-77-13-47.png
1781 data_cropped/67-12-02-30-24-48-86.png
1782 data_cropped/79-95-05-59-45-48-94.png
1783 data_cropped/84-24-05-68-86-56-40.png
1784 data_cropped/03-01-06-02-01-02-12.png
1785 data_cropped/16-72-06-77-95-96-98.png
1786 data_cropped/44-88-01-78-58-95-54.png
1787 data_cropped/55-93-05-95-96-98-60.png
1788 data_cropped/57-47-02-38-71-10-45.png
1789 data_cropped/49-94-02-91-27-55-40.png
1790 data_cropped/78-65-05-16-46-17-72.png
1791 data_cropped/92-75-02-33-53-45-82.png
1792 data_cropped/63-89-06-67-48-37-85.png
1793 data_cropped/48-11-04-39-47-69-71.png
1794 data_cropped/93-68-04-78-70-71-14.png
1795 data_cropped/72-83-01-34-69-29-81.png
1796 data_cropped/19-75-03-77-18-12-84.png
1797 data_cropped/27-55-06-98-65-62-86.png
1798 data_cropped/18-64-05-28-69-12-73.png
1799 data_cropped/44-50-01-69-34-30-20.png
1800 data_cropped/62-89-03-67-41-97-45.png
1801 data_cropped/03-01-06-02-02-02-10.png
1802 data_cropped/84-90-06-75-86-92-83.png
1803 data_c

1978 data_cropped/78-46-02-44-87-80-29.png
1979 data_cropped/36-91-05-18-86-90-95.png
1980 data_cropped/83-16-06-79-94-29-26.png
1981 data_cropped/19-34-02-36-37-39-82.png
1982 data_cropped/71-57-05-64-56-25-73.png
1983 data_cropped/68-40-03-15-55-57-94.png
1984 data_cropped/16-91-02-64-89-13-36.png
1985 data_cropped/59-92-03-62-97-31-73.png
1986 data_cropped/76-68-01-49-95-89-45.png
1987 data_cropped/30-71-06-15-67-26-63.png
1988 data_cropped/10-88-05-78-77-51-22.png
1989 data_cropped/36-88-06-55-52-31-49.png
1990 data_cropped/91-98-06-13-26-90-15.png
1991 data_cropped/73-30-06-54-86-55-65.png
1992 data_cropped/61-44-01-75-21-89-37.png
1993 data_cropped/26-82-03-81-96-68-17.png
1994 data_cropped/30-90-05-32-97-74-65.png
1995 data_cropped/39-89-05-79-36-76-58.png
1996 data_cropped/18-87-03-11-94-69-97.png
1997 data_cropped/03-01-04-02-02-02-17.png
1998 data_cropped/46-63-03-91-25-30-75.png
1999 data_cropped/72-40-04-95-39-15-28.png
2000 data_cropped/85-93-04-48-68-71-81.png
2001 data_c

2177 data_cropped/36-31-06-60-91-51-58.png
2178 data_cropped/85-20-05-23-87-72-80.png
2179 data_cropped/38-57-06-83-62-48-29.png
2180 data_cropped/45-36-02-83-96-24-97.png
2181 data_cropped/31-67-03-42-62-60-49.png
2182 data_cropped/65-61-03-48-69-56-12.png
2183 data_cropped/73-90-06-45-41-96-69.png
2184 data_cropped/46-96-03-37-52-56-22.png
2185 data_cropped/13-72-06-53-87-64-79.png
2186 data_cropped/55-84-01-90-18-36-19.png
2187 data_cropped/88-47-01-66-64-30-63.png
2188 data_cropped/97-48-02-60-20-45-58.png
2189 data_cropped/15-20-03-71-45-92-33.png
2190 data_cropped/12-36-01-80-24-26-61.png
2191 data_cropped/41-68-01-53-50-49-51.png
2192 data_cropped/72-65-04-27-87-69-98.png
2193 data_cropped/22-10-04-76-37-50-75.png
2194 data_cropped/16-64-01-89-39-21-76.png
2195 data_cropped/48-24-01-83-94-20-61.png
2196 data_cropped/44-92-05-34-41-85-55.png
2197 data_cropped/97-45-05-22-91-94-51.png
2198 data_cropped/71-19-01-53-11-78-72.png
2199 data_cropped/98-86-02-60-78-75-32.png
2200 data_c

2376 data_cropped/58-38-04-54-60-10-26.png
2377 data_cropped/15-21-02-42-30-59-96.png
2378 data_cropped/78-65-05-15-48-28-94.png
2379 data_cropped/66-10-02-68-46-80-87.png
2380 data_cropped/39-77-02-63-72-28-88.png
2381 data_cropped/75-89-02-28-56-43-41.png
2382 data_cropped/76-51-02-91-18-26-90.png
2383 data_cropped/15-55-04-69-21-34-91.png
2384 data_cropped/81-93-03-24-65-79-55.png
2385 data_cropped/69-34-04-86-57-16-37.png
2386 data_cropped/72-20-01-22-52-23-94.png
2387 data_cropped/67-88-02-48-10-94-59.png
2388 data_cropped/98-23-05-49-14-74-84.png
2389 data_cropped/13-80-04-65-94-28-36.png
2390 data_cropped/17-67-05-68-26-98-33.png
2391 data_cropped/87-90-01-50-26-14-65.png
2392 data_cropped/52-76-03-75-32-43-12.png
2393 data_cropped/03-01-05-02-01-02-12.png
2394 data_cropped/76-95-04-81-59-68-33.png
2395 data_cropped/12-64-03-93-17-26-42.png
2396 data_cropped/84-53-01-72-55-76-54.png
2397 data_cropped/16-91-02-44-42-68-84.png
2398 data_cropped/66-67-02-91-36-42-46.png
2399 data_c

2573 data_cropped/03-01-04-02-01-01-13.png
2574 data_cropped/63-61-02-65-66-75-70.png
2575 data_cropped/24-81-02-51-47-98-83.png
2576 data_cropped/41-75-05-91-16-58-13.png
2577 data_cropped/41-69-05-20-74-47-30.png
2578 data_cropped/92-34-05-75-86-67-30.png
2579 data_cropped/56-72-03-29-25-12-10.png
2580 data_cropped/51-40-02-73-19-21-53.png
2581 data_cropped/63-11-01-42-69-29-82.png
2582 data_cropped/72-95-01-22-79-84-10.png
2583 data_cropped/24-27-03-95-89-43-58.png
2584 data_cropped/87-78-03-45-30-53-82.png
2585 data_cropped/39-51-05-44-88-60-30.png
2586 data_cropped/87-76-01-19-92-64-22.png
2587 data_cropped/31-48-05-58-37-93-60.png
2588 data_cropped/72-61-03-56-40-42-90.png
2589 data_cropped/90-70-01-97-64-92-91.png
2590 data_cropped/97-96-05-54-27-41-81.png
2591 data_cropped/20-19-03-57-62-94-76.png
2592 data_cropped/82-53-03-29-11-66-60.png
2593 data_cropped/03-01-03-01-01-02-02.png
2594 data_cropped/95-25-06-43-71-17-24.png
2595 data_cropped/11-23-04-79-85-91-35.png
2596 data_c

In [52]:
X_name = 'X_mel_in_mfcc_test_par.joblib'
y_name = 'y_mel_in_mfcc_test_par.joblib'
save_dir = 'mel_gd_joblibs'

savedX = joblib.dump(data, os.path.join(save_dir, X_name))
savedy = joblib.dump(label, os.path.join(save_dir, y_name))

In [53]:
import keras

In [54]:
input_ = keras.Input(shape=(40,1))
model1 = Conv1D(64, 5,padding='same')(input_)
model1 = Activation('relu')(model1)
model1 = Dropout(0.1)(model1)
model1 = MaxPooling1D(pool_size=(4))(model1)
model1= Conv1D(128, 5,padding='same')(model1)
model1= Activation('relu')(model1)
model1 = Dropout(0.1)(model1)
model1 = MaxPooling1D(pool_size=(4))(model1)
model1= Conv1D(256, 5,padding='same')(model1)
model1 = Activation('relu')(model1)
model1 = Dropout(0.1)(model1)
model1 = Flatten()(model1)

In [55]:
model_1 = keras.Model(input_, model1, name="model1")

In [56]:
model_1.summary()

Model: "model1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 40, 1)]           0         
_________________________________________________________________
conv1d (Conv1D)              (None, 40, 64)            384       
_________________________________________________________________
activation (Activation)      (None, 40, 64)            0         
_________________________________________________________________
dropout (Dropout)            (None, 40, 64)            0         
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 10, 64)            0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 10, 128)           41088     
_________________________________________________________________
activation_1 (Activation)    (None, 10, 128)           0    

In [57]:
# example of loading the resnet50 model
from keras.applications.resnet50 import ResNet50
# load model
model_2 = ResNet50(weights='imagenet', include_top=False,input_shape=(224, 224, 3))

In [58]:
model_2.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [59]:
for layer in model_2.layers[:]:
  layer.trainable = False

for layer in model_2.layers[:]:
  print(layer, layer.trainable)

<tensorflow.python.keras.engine.input_layer.InputLayer object at 0x000002734614B610> False
<tensorflow.python.keras.layers.convolutional.ZeroPadding2D object at 0x000002752502CD00> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x000002752502C580> False
<tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 0x0000027348F0FF10> False
<tensorflow.python.keras.layers.core.Activation object at 0x0000027525032F40> False
<tensorflow.python.keras.layers.convolutional.ZeroPadding2D object at 0x0000027525032730> False
<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x00000275250329A0> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x000002752507D940> False
<tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 0x000002752507DBE0> False
<tensorflow.python.keras.layers.core.Activation object at 0x00000275250AB670> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x000002752

In [60]:
model_3 = Sequential()
model_3.add(model_2)
model_3.add(Flatten())
model_3.add(Dense(512, activation = 'relu'))
model_3.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Functional)        (None, 7, 7, 2048)        23587712  
_________________________________________________________________
flatten_1 (Flatten)          (None, 100352)            0         
_________________________________________________________________
dense (Dense)                (None, 512)               51380736  
Total params: 74,968,448
Trainable params: 51,380,736
Non-trainable params: 23,587,712
_________________________________________________________________


In [61]:
from keras.layers import concatenate

In [62]:
combinedInput = concatenate([model_1.output, model_3.output]) 

In [63]:
new_model = Flatten()(combinedInput)
new_model = Dense(1024, activation="relu")(new_model)
new_model = Dense(6, activation="softmax")(new_model)

In [76]:
model = Model(inputs=[model_1.input, model_3.input], outputs=new_model)

In [77]:
model.summary()

Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 40, 1)]      0                                            
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, 40, 64)       384         input_1[0][0]                    
__________________________________________________________________________________________________
activation (Activation)         (None, 40, 64)       0           conv1d[0][0]                     
__________________________________________________________________________________________________
dropout (Dropout)               (None, 40, 64)       0           activation[0][0]                 
_______________________________________________________________________________________

In [66]:
X_mfcc_train = joblib.load('mel_gd_joblibs/X_mfcc_train_par.joblib')
X_mel_train = joblib.load('mel_gd_joblibs/X_mel_in_mfcc_train_par.joblib')
X_mfcc_test = joblib.load('mel_gd_joblibs/X_mfcc_test_par.joblib')
X_mel_test = joblib.load('mel_gd_joblibs/X_mel_in_mfcc_test_par.joblib')
y_train = joblib.load('mel_gd_joblibs/y_mfcc_train_par.joblib')
y_test = joblib.load('mel_gd_joblibs/y_mfcc_test_par.joblib')

In [67]:
print(X_mel_train.shape, X_mfcc_train.shape, X_mel_test.shape, X_mfcc_test.shape, y_train.shape, y_test.shape)

(10974, 224, 224, 3) (10974, 40) (2744, 224, 224, 3) (2744, 40) (10974,) (2744,)


In [78]:
from keras.optimizers import SGD, RMSprop
opt = RMSprop(lr=0.00005, rho=0.9, epsilon=1e-07, decay=0.0)

In [79]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [80]:
from keras.callbacks import ModelCheckpoint

In [81]:
EPOCHS = 10

checkpoint_filepath = 'Resnet50_mfcc_cnn_mel.h5'
model_checkpoint_callback = ModelCheckpoint(
    filepath= checkpoint_filepath,
    save_weights_only=False,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

model.fit([X_mfcc_train, X_mel_train],y_train, batch_size = 16, epochs = EPOCHS, validation_data = ([X_mfcc_test, X_mel_test], y_test),callbacks=[model_checkpoint_callback])
    
    
    

Epoch 1/10
686/686 [==============================] - 685s 998ms/step - loss: 0.4292 - accuracy: 0.8462 - val_loss: 0.8338 - val_accuracy: 0.7172
Epoch 2/10
686/686 [==============================] - 661s 963ms/step - loss: 0.2969 - accuracy: 0.8934 - val_loss: 1.0174 - val_accuracy: 0.6906
Epoch 3/10
686/686 [==============================] - 665s 969ms/step - loss: 0.2134 - accuracy: 0.9251 - val_loss: 1.3561 - val_accuracy: 0.6953
Epoch 4/10
686/686 [==============================] - 666s 971ms/step - loss: 0.1500 - accuracy: 0.9471 - val_loss: 1.2139 - val_accuracy: 0.7103
Epoch 5/10
686/686 [==============================] - 666s 971ms/step - loss: 0.1062 - accuracy: 0.9636 - val_loss: 1.3982 - val_accuracy: 0.7161
Epoch 6/10
686/686 [==============================] - 666s 971ms/step - loss: 0.0832 - accuracy: 0.9714 - val_loss: 1.9683 - val_accuracy: 0.6808
Epoch 7/10
686/686 [==============================] - 669s 976ms/step - loss: 0.0674 - accuracy: 0.9778 - val_loss: 1.6837 -

In [82]:
predictions = model.predict([X_mfcc_test, X_mel_test])
predictions

array([[1.26372175e-08, 1.15059481e-11, 9.99299407e-01, 4.31331533e-12,
        7.00491713e-04, 9.74588232e-08],
       [3.94817558e-04, 5.85386455e-01, 2.30040634e-03, 2.27691010e-01,
        2.18832330e-03, 1.82038963e-01],
       [2.63037549e-07, 6.82929695e-01, 1.94693096e-02, 8.95884782e-02,
        2.06132323e-01, 1.87987636e-03],
       ...,
       [5.08822186e-26, 2.73515368e-22, 2.66142372e-33, 1.00000000e+00,
        1.80915645e-31, 4.76720399e-30],
       [2.10003244e-19, 3.33544588e-20, 8.95718299e-29, 1.00000000e+00,
        1.29424720e-33, 5.43319159e-17],
       [1.53769697e-09, 8.27551261e-03, 2.13644642e-04, 2.28774923e-07,
        9.91510570e-01, 2.69658269e-08]], dtype=float32)

In [83]:
predictions.shape

(2744, 6)

In [84]:
predictions = np.argmax(predictions,axis=1)

In [85]:
predictions.shape

(2744,)

In [86]:
new_Ytest = y_test.astype(int)
new_Ytest

array([2, 4, 1, ..., 3, 3, 3])

In [87]:
predictions

array([2, 1, 1, ..., 3, 3, 4], dtype=int64)

In [88]:
from sklearn.metrics import classification_report
report = classification_report(new_Ytest, predictions)
print(report)

              precision    recall  f1-score   support

           0       0.76      0.78      0.77       411
           1       0.66      0.75      0.70       458
           2       0.74      0.67      0.71       455
           3       0.88      0.77      0.82       446
           4       0.71      0.71      0.71       500
           5       0.68      0.70      0.69       474

    accuracy                           0.73      2744
   macro avg       0.74      0.73      0.73      2744
weighted avg       0.74      0.73      0.73      2744



In [89]:

loss, acc = model.evaluate([X_mfcc_test,X_mel_test], new_Ytest)
print("Model accuracy: {:5.2f}%".format(100*acc))


86/86 [==============================] - 92s 1s/step - loss: 1.9103 - accuracy: 0.7314
Model accuracy: 73.14%


In [90]:
label_conversion =  {'W': '4' , 'E': '6'  , 'A': '5'  , 'F': '2'   ,'T': '3'   , 'N': '1'}

In [91]:
path = []
label= []
for fil in os.listdir('Emo_datacrop_mel'):
    if fil.split('.')[0][-2] != 'L':
        path.append('Emodb/'+fil.split('.')[0]+'.wav')
        for key, val in label_conversion.items():
            if fil.split('.')[0][-2] == key:
                label.append(val)

In [92]:
len(path)

454

In [93]:
path

['Emodb/03a01Fa.wav',
 'Emodb/03a01Nc.wav',
 'Emodb/03a01Wa.wav',
 'Emodb/03a02Fc.wav',
 'Emodb/03a02Nc.wav',
 'Emodb/03a02Ta.wav',
 'Emodb/03a02Wb.wav',
 'Emodb/03a02Wc.wav',
 'Emodb/03a04Ad.wav',
 'Emodb/03a04Fd.wav',
 'Emodb/03a04Nc.wav',
 'Emodb/03a04Ta.wav',
 'Emodb/03a04Wc.wav',
 'Emodb/03a05Aa.wav',
 'Emodb/03a05Fc.wav',
 'Emodb/03a05Nd.wav',
 'Emodb/03a05Tc.wav',
 'Emodb/03a05Wa.wav',
 'Emodb/03a05Wb.wav',
 'Emodb/03a07Fa.wav',
 'Emodb/03a07Fb.wav',
 'Emodb/03a07Nc.wav',
 'Emodb/03a07Wc.wav',
 'Emodb/03b01Fa.wav',
 'Emodb/03b01Nb.wav',
 'Emodb/03b01Td.wav',
 'Emodb/03b01Wa.wav',
 'Emodb/03b01Wc.wav',
 'Emodb/03b02Aa.wav',
 'Emodb/03b02Na.wav',
 'Emodb/03b02Tb.wav',
 'Emodb/03b02Wb.wav',
 'Emodb/03b03Nb.wav',
 'Emodb/03b03Tc.wav',
 'Emodb/03b03Wc.wav',
 'Emodb/03b09Nc.wav',
 'Emodb/03b09Tc.wav',
 'Emodb/03b09Wa.wav',
 'Emodb/03b10Ab.wav',
 'Emodb/03b10Ec.wav',
 'Emodb/03b10Na.wav',
 'Emodb/03b10Nc.wav',
 'Emodb/03b10Wb.wav',
 'Emodb/03b10Wc.wav',
 'Emodb/08a01Ab.wav',
 'Emodb/08

In [ ]:
'03a01Fa.png'.split('.')[0][-2]

In [94]:
path1 = []
label1 = []

for fil in os.listdir('Emo_datacrop_mel'):
    if fil.split('.')[0][-2] != 'L':
        path1.append('Emo_datacrop_mel/'+fil)
        for key, val in label_conversion.items():
            if fil.split('.')[0][-2] == key:
                label1.append(val)

In [95]:
len(path1)

454

In [96]:
path1

['Emo_datacrop_mel/03a01Fa.png',
 'Emo_datacrop_mel/03a01Nc.png',
 'Emo_datacrop_mel/03a01Wa.png',
 'Emo_datacrop_mel/03a02Fc.png',
 'Emo_datacrop_mel/03a02Nc.png',
 'Emo_datacrop_mel/03a02Ta.png',
 'Emo_datacrop_mel/03a02Wb.png',
 'Emo_datacrop_mel/03a02Wc.png',
 'Emo_datacrop_mel/03a04Ad.png',
 'Emo_datacrop_mel/03a04Fd.png',
 'Emo_datacrop_mel/03a04Nc.png',
 'Emo_datacrop_mel/03a04Ta.png',
 'Emo_datacrop_mel/03a04Wc.png',
 'Emo_datacrop_mel/03a05Aa.png',
 'Emo_datacrop_mel/03a05Fc.png',
 'Emo_datacrop_mel/03a05Nd.png',
 'Emo_datacrop_mel/03a05Tc.png',
 'Emo_datacrop_mel/03a05Wa.png',
 'Emo_datacrop_mel/03a05Wb.png',
 'Emo_datacrop_mel/03a07Fa.png',
 'Emo_datacrop_mel/03a07Fb.png',
 'Emo_datacrop_mel/03a07Nc.png',
 'Emo_datacrop_mel/03a07Wc.png',
 'Emo_datacrop_mel/03b01Fa.png',
 'Emo_datacrop_mel/03b01Nb.png',
 'Emo_datacrop_mel/03b01Td.png',
 'Emo_datacrop_mel/03b01Wa.png',
 'Emo_datacrop_mel/03b01Wc.png',
 'Emo_datacrop_mel/03b02Aa.png',
 'Emo_datacrop_mel/03b02Na.png',
 'Emo_data

In [97]:
emodf_mfcc = pd.DataFrame({'path' : path, 'label' : label})
emodf_mfcc.head()

,path,label
0,Emodb/03a01Fa.wav,2
1,Emodb/03a01Nc.wav,1
2,Emodb/03a01Wa.wav,4
3,Emodb/03a02Fc.wav,2
4,Emodb/03a02Nc.wav,1


In [98]:
emodf_mel = pd.DataFrame({'path' : path1, 'label' : label1})
emodf_mel.head()

,path,label
0,Emo_datacrop_mel/03a01Fa.png,2
1,Emo_datacrop_mel/03a01Nc.png,1
2,Emo_datacrop_mel/03a01Wa.png,4
3,Emo_datacrop_mel/03a02Fc.png,2
4,Emo_datacrop_mel/03a02Nc.png,1


In [99]:
data = []
label =[]
    
for lab, img in enumerate(emodf_mel.path):
  try:
    image = load_img(img, target_size=(224, 224))
    # convert the image pixels to a numpy array 
    image = img_to_array(image) 
    # reshape data for the model
    image = preprocess_input(image)
    data.append(image)
    label.append(int(emodf_mel.label[lab])-1)
    print(lab,emodf['path'][lab])
  except:
    print("Error loading image", lab)

    

#Converting lists into numpy arrays
data = np.array(data)
label = np.array(label)

print(data.shape, label.shape)

Error loading image 0
Error loading image 1
Error loading image 2
Error loading image 3
Error loading image 4
Error loading image 5
Error loading image 6
Error loading image 7
Error loading image 8
Error loading image 9
Error loading image 10
Error loading image 11
Error loading image 12
Error loading image 13
Error loading image 14
Error loading image 15
Error loading image 16
Error loading image 17
Error loading image 18
Error loading image 19
Error loading image 20
Error loading image 21
Error loading image 22
Error loading image 23
Error loading image 24
Error loading image 25
Error loading image 26
Error loading image 27
Error loading image 28
Error loading image 29
Error loading image 30
Error loading image 31
Error loading image 32
Error loading image 33
Error loading image 34
Error loading image 35
Error loading image 36
Error loading image 37
Error loading image 38
Error loading image 39
Error loading image 40
Error loading image 41
Error loading image 42
Error loading image 4

Error loading image 351
Error loading image 352
Error loading image 353
Error loading image 354
Error loading image 355
Error loading image 356
Error loading image 357
Error loading image 358
Error loading image 359
Error loading image 360
Error loading image 361
Error loading image 362
Error loading image 363
Error loading image 364
Error loading image 365
Error loading image 366
Error loading image 367
Error loading image 368
Error loading image 369
Error loading image 370
Error loading image 371
Error loading image 372
Error loading image 373
Error loading image 374
Error loading image 375
Error loading image 376
Error loading image 377
Error loading image 378
Error loading image 379
Error loading image 380
Error loading image 381
Error loading image 382
Error loading image 383
Error loading image 384
Error loading image 385
Error loading image 386
Error loading image 387
Error loading image 388
Error loading image 389
Error loading image 390
Error loading image 391
Error loading im

In [ ]:
X_name = 'X_emo_mel_test.joblib'
y_name = 'y_emo_mel_test.joblib'
save_dir = 'mel_gd_joblibs'

savedX = joblib.dump(data, os.path.join(save_dir, X_name))
savedy = joblib.dump(label, os.path.join(save_dir, y_name))

###### Mfcc of the EmoDb Dataset for testing to left side of the model.

In [100]:
import time
import os
lst = []

start_time = time.time()

for lab, audio in enumerate(emodf_mfcc.path):
    try:
        #Load librosa array, obtain mfcss, store the file and the mcss information in a new array
        X, sample_rate = librosa.load(audio, res_type='kaiser_fast')
        mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T,axis=0) 
        # The instruction below converts the labels (from 1 to 8) to a series from 0 to 7
        # This is because our predictor needs to start from 0 otherwise it will try to predict also 0.
        file = int(emodf_mfcc.label[lab]) - 1 
        arr = mfccs, file
        lst.append(arr)
        print(audio, lab, emodf_mfcc.label[lab])
        # If the file is not valid, skip it
    except ValueError:
        continue

print("--- Data loaded. Loading time: %s seconds ---" % (time.time() - start_time))

Emodb/03a01Fa.wav 0 2
Emodb/03a01Nc.wav 1 1
Emodb/03a01Wa.wav 2 4
Emodb/03a02Fc.wav 3 2
Emodb/03a02Nc.wav 4 1
Emodb/03a02Ta.wav 5 3
Emodb/03a02Wb.wav 6 4
Emodb/03a02Wc.wav 7 4
Emodb/03a04Ad.wav 8 5
Emodb/03a04Fd.wav 9 2
Emodb/03a04Nc.wav 10 1
Emodb/03a04Ta.wav 11 3
Emodb/03a04Wc.wav 12 4
Emodb/03a05Aa.wav 13 5
Emodb/03a05Fc.wav 14 2
Emodb/03a05Nd.wav 15 1
Emodb/03a05Tc.wav 16 3
Emodb/03a05Wa.wav 17 4
Emodb/03a05Wb.wav 18 4
Emodb/03a07Fa.wav 19 2
Emodb/03a07Fb.wav 20 2
Emodb/03a07Nc.wav 21 1
Emodb/03a07Wc.wav 22 4
Emodb/03b01Fa.wav 23 2
Emodb/03b01Nb.wav 24 1
Emodb/03b01Td.wav 25 3
Emodb/03b01Wa.wav 26 4
Emodb/03b01Wc.wav 27 4
Emodb/03b02Aa.wav 28 5
Emodb/03b02Na.wav 29 1
Emodb/03b02Tb.wav 30 3
Emodb/03b02Wb.wav 31 4
Emodb/03b03Nb.wav 32 1
Emodb/03b03Tc.wav 33 3
Emodb/03b03Wc.wav 34 4
Emodb/03b09Nc.wav 35 1
Emodb/03b09Tc.wav 36 3
Emodb/03b09Wa.wav 37 4
Emodb/03b10Ab.wav 38 5
Emodb/03b10Ec.wav 39 6
Emodb/03b10Na.wav 40 1
Emodb/03b10Nc.wav 41 1
Emodb/03b10Wb.wav 42 4
Emodb/03b10Wc.wav 43 

Emodb/15a02Ac.wav 354 5
Emodb/15a02Ea.wav 355 6
Emodb/15a02Na.wav 356 1
Emodb/15a02Ta.wav 357 3
Emodb/15a02Wb.wav 358 4
Emodb/15a02Wd.wav 359 4
Emodb/15a04Ab.wav 360 5
Emodb/15a04Ac.wav 361 5
Emodb/15a04Fd.wav 362 2
Emodb/15a04Nc.wav 363 1
Emodb/15a04Wa.wav 364 4
Emodb/15a04Wb.wav 365 4
Emodb/15a05Eb.wav 366 6
Emodb/15a05Fb.wav 367 2
Emodb/15a05Na.wav 368 1
Emodb/15a05Wa.wav 369 4
Emodb/15a07Ac.wav 370 5
Emodb/15a07Eb.wav 371 6
Emodb/15a07Fa.wav 372 2
Emodb/15a07Fb.wav 373 2
Emodb/15a07Nc.wav 374 1
Emodb/15b01Ec.wav 375 6
Emodb/15b01Na.wav 376 1
Emodb/15b01Wc.wav 377 4
Emodb/15b02Aa.wav 378 5
Emodb/15b02Nd.wav 379 1
Emodb/15b02Tc.wav 380 3
Emodb/15b02Wa.wav 381 4
Emodb/15b02Wc.wav 382 4
Emodb/15b03Aa.wav 383 5
Emodb/15b03Nb.wav 384 1
Emodb/15b03Tc.wav 385 3
Emodb/15b03Wa.wav 386 4
Emodb/15b03Wb.wav 387 4
Emodb/15b09Ac.wav 388 5
Emodb/15b09Fa.wav 389 2
Emodb/15b09Nb.wav 390 1
Emodb/15b09Ta.wav 391 3
Emodb/15b09Wb.wav 392 4
Emodb/15b10Ac.wav 393 5
Emodb/15b10Nb.wav 394 1
Emodb/15b10Nc.wa

In [101]:
len(lst)

454

In [102]:
# Creating X and y: zip makes a list of all the first elements, and a list of all the second elements.
X, y = zip(*lst)
import numpy as np
X = np.asarray(X)
y = np.asarray(y)
X.shape, y.shape

((454, 40), (454,))

In [103]:
X_name = 'X_emo_mfcc_test.joblib'
y_name = 'y_emo_mfcc_test.joblib'
save_dir = 'mel_gd_joblibs'

savedX = joblib.dump(X, os.path.join(save_dir, X_name))
savedy = joblib.dump(y, os.path.join(save_dir, y_name))

In [104]:
X_emo_mel = joblib.load('mel_gd_joblibs/X_emo_mel_test.joblib')
X_emo_mfcc = joblib.load('mel_gd_joblibs/X_emo_mfcc_test.joblib')
y_emo_mfcc = joblib.load('mel_gd_joblibs/y_emo_mfcc_test.joblib')

In [105]:
X_emo_mel.shape, y_emo_mfcc.shape, X_emo_mfcc.shape

((454, 224, 224, 3), (454,), (454, 40))

In [106]:
predictions = model.predict([X_emo_mfcc, X_emo_mel])
predictions

array([[6.83162920e-03, 1.06693376e-02, 5.58721888e-11, 9.71946836e-01,
        3.76714979e-07, 1.05518410e-02],
       [1.13227707e-03, 2.22148880e-01, 7.30904595e-14, 7.76699781e-01,
        2.57478969e-11, 1.91172912e-05],
       [2.56297650e-10, 2.95860923e-06, 1.87364118e-17, 9.99997020e-01,
        1.27687195e-13, 2.68384226e-09],
       ...,
       [7.64157465e-15, 1.14144579e-11, 1.62924513e-12, 3.12330804e-12,
        5.12201447e-13, 1.00000000e+00],
       [1.53933601e-14, 1.00000000e+00, 7.10735751e-18, 2.06736342e-12,
        4.69889741e-17, 1.94261361e-18],
       [6.70640750e-15, 9.99971986e-01, 1.41238915e-14, 2.80276709e-05,
        2.52030236e-17, 6.94609214e-12]], dtype=float32)

In [107]:
predictions.shape

(454, 6)

In [108]:
predictions = np.argmax(predictions,axis=1)

In [109]:
predictions.shape

(454,)

In [110]:
from sklearn.metrics import classification_report
report = classification_report(y_emo_mfcc, predictions)
print(report)

              precision    recall  f1-score   support

           0       0.80      0.30      0.44        79
           1       0.29      0.42      0.34        71
           2       0.33      0.02      0.03        62
           3       0.55      0.60      0.57       127
           4       0.62      0.14      0.24        69
           5       0.24      0.85      0.37        46

    accuracy                           0.40       454
   macro avg       0.47      0.39      0.33       454
weighted avg       0.50      0.40      0.37       454



In [111]:

loss, acc = model.evaluate([X_emo_mfcc, X_emo_mel], y_emo_mfcc)
print("Model accuracy: {:5.2f}%".format(100*acc))


15/15 [==============================] - 14s 961ms/step - loss: 8.2339 - accuracy: 0.3965
Model accuracy: 39.65%
